# Introduction to financial technical analysis

# 📌 Objectives

By the end of this notebook, students will be able to:

1. **Access and Manipulate Financial Data:**
   - Use the `yfinance` library to retrieve historical stock prices for all companies in the S&P 500 index.

2. **Compute Key Technical Indicators:**
   - Calculate 50-day and 200-day moving averages for each stock in the index.

3. **Detect Trading Signals:**
   - Identify Golden Crosses and Death Crosses as described in technical analysis literature.

4. **Analyze Recent Market Behavior:**
   - Isolate and examine stocks that experienced technical signals (crosses) in the last 14 days.

5. **Visualize Price Trends and Volatility:**
   - Plot historical price movements along with moving averages, and compute volatility around signal dates.

6. **Interpret Technical Indicators in Context:**
   - Reflect on what Golden and Death Crosses signify and how traders may respond to them.

7. **Evaluate Strategy Viability:**
   - Discuss the strengths and limitations of using moving averages as a standalone trading strategy.

8. **Connect Technical Analysis to Broader Market Intelligence:**
   - Explore how sentiment analysis and news (covered in later sections) could complement technical signals.

9. **Develop Critical Thinking About Signal Reliability:**
   - Assess potential risks of false positives and propose improvements or filters to enhance signal accuracy.

10. **Engage in Strategic Reflection:**
    - Answer analytical questions aimed at understanding the utility, risks, and presentation of the strategy to a professional audience.


## Import and install librairies

In [1]:
# %pip install pandas
# %pip install yfinance
# %pip install lxml
# %pip install matplotlib


In [2]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt
import numpy as np



## Get the list of stocks in the S&P 500 

In [3]:
# Read and print the stock tickers that make up S&P500
df_tickers = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
print(df_tickers.head())

  Symbol             Security             GICS Sector  \
0    MMM                   3M             Industrials   
1    AOS          A. O. Smith             Industrials   
2    ABT  Abbott Laboratories             Health Care   
3   ABBV               AbbVie             Health Care   
4    ACN            Accenture  Information Technology   

                GICS Sub-Industry    Headquarters Location  Date added  \
0        Industrial Conglomerates    Saint Paul, Minnesota  1957-03-04   
1               Building Products     Milwaukee, Wisconsin  2017-07-26   
2           Health Care Equipment  North Chicago, Illinois  1957-03-04   
3                   Biotechnology  North Chicago, Illinois  2012-12-31   
4  IT Consulting & Other Services          Dublin, Ireland  2011-07-06   

       CIK      Founded  
0    66740         1902  
1    91142         1916  
2     1800         1888  
3  1551152  2013 (1888)  
4  1467373         1989  


In [4]:
display(df_tickers)

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [5]:
ticker_list = df_tickers['Symbol'].tolist()

## Get the closing price of all 500 stocks in the S&P 500 Index
Use the yfinance library to retrieve the close price of all 500 stocks in the index between 2024-05-01 and 2025-05-01
https://ranaroussi.github.io/yfinance/reference/yfinance.stock.html

In [6]:
start_date = '2024-05-01'
end_date = '2025-05-01'

In [55]:
df_close = yf.Tickers(ticker_list).history(start=start_date, end=end_date, period=None)['Close']

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-05-01 -> 2025-05-01)')


In [56]:
# df_close has a row for each date and a column for each stock ticker.
# It is indexed by the date, and the value for each cell is the closing price of the stock on that date.
display(df_close)

Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WY,WYNN,XEL,XOM,XYL,XYZ,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2024-05-01,137.376282,168.283661,154.714767,156.160004,103.735458,91.096085,293.442139,469.390015,189.859940,55.668903,...,29.172792,91.453209,51.484390,111.217087,129.094406,66.839996,132.078354,118.202553,309.049988,156.223297
2024-05-02,136.217377,171.991287,153.844193,158.330002,103.374344,90.497009,295.092743,476.570007,192.405914,56.603558,...,29.955238,93.958237,51.493961,111.418373,134.052750,70.300003,131.824631,117.115036,312.709991,164.827896
2024-05-03,138.129074,182.279160,156.695114,159.710007,103.354828,89.774330,298.403900,486.179993,195.480652,56.431889,...,30.032518,95.378403,51.934326,111.188332,135.255371,69.470001,131.112137,120.278717,309.589996,164.670197
2024-05-06,139.278091,180.619171,155.681030,162.000000,103.101074,92.370285,300.712830,493.589996,199.093948,57.404686,...,30.022860,96.522438,52.039635,111.907211,137.217010,73.529999,132.907913,119.191200,315.790009,163.566269
2024-05-07,139.931839,181.305008,155.508820,159.809998,103.618340,93.387741,305.232452,492.269989,199.338745,58.739899,...,29.800680,95.891251,52.671459,111.351273,138.035187,71.599998,132.644409,120.051315,317.869995,166.030365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-24,106.795090,208.097107,178.825470,121.709999,128.801849,92.099998,291.410004,360.910004,194.345154,48.284031,...,25.294355,81.371361,69.709930,107.638626,115.241966,57.500000,146.859604,101.237473,244.080002,151.792435
2025-04-25,106.056648,209.005920,184.466751,122.510002,128.274216,90.680000,293.390015,367.720001,193.708054,47.759418,...,24.609919,81.900002,68.430580,107.579178,115.730400,58.090000,146.580978,101.247452,246.240005,152.958542
2025-04-28,106.645409,209.864792,190.692963,123.300003,128.951172,91.190002,293.250000,368.619995,192.841995,47.561451,...,25.363791,81.989769,69.005791,107.638626,115.550972,58.320000,147.028778,101.496803,243.490005,153.058212


## Identify Golden and Death Crosses

### Get Moving Averages 50 days and 200 days

In [95]:
# We start by defining a function to compute the moving averages for a single stock.
def compute_moving_averages(values: np.ndarray, window: int) -> np.ndarray:
    """
    Compute the moving averages of a given array of values with the specified window size.
    """
    result = np.full_like(values, np.nan)
    for i in range(window, len(values)):
        result[i] = np.mean(values[i-window:i])

    return result

In [96]:
# We now compute the moving averages for all stocks
mv_avgs_50 = {}

for ticker in ticker_list:
    stock = df_close[ticker]
    mv_avg_50 = compute_moving_averages(stock.values, 50)
    mv_avgs_50[ticker] = mv_avg_50

In [105]:
# And convert it to a DataFrame, indexed by the date

df_ma50 = pd.DataFrame(mv_avgs_50, index=df_close.index)

# We skip the first 50 rows because they will be NaN, since there is not enough data to compute the moving average for those dates.
display(df_ma50.iloc[51:55])

,MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,AFL,A,...,WMB,WTW,WDAY,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2024-07-16,98.363778,82.086964,101.650910,158.097099,294.541234,502.910001,162.9840,18.518137,86.218276,136.155668,...,39.552557,253.314334,229.0936,90.837969,52.200766,137.285973,132.570661,312.963799,111.715038,169.825461
2024-07-17,98.504225,82.252492,101.599086,158.287370,295.053825,504.709400,163.6118,18.493878,86.414030,136.131746,...,39.634545,253.564847,228.7892,90.658511,52.213965,137.397157,132.462982,313.424800,111.502259,170.134557
2024-07-18,98.664117,82.366967,101.586112,158.537831,295.512036,506.247601,163.7884,18.460143,86.635951,136.018781,...,39.719841,253.887487,228.3062,90.430281,52.245242,137.439617,132.377595,313.750800,111.297348,170.429853
2024-07-19,98.822757,82.442957,101.487752,158.728825,295.973410,507.512800,163.7882,18.426029,86.832762,135.852470,...,39.809851,254.172438,227.9196,90.150694,52.283310,137.423868,132.236285,313.902400,111.120926,170.700042


In [106]:
# Repeat the whole process for the 200-day moving average

mv_avgs_200 = {}
for ticker in ticker_list:
    stock = df_close[ticker]
    mv_avg_200 = compute_moving_averages(stock.values, 200)
    mv_avgs_200[ticker] = mv_avg_200

df_ma200 = pd.DataFrame(mv_avgs_200, index=df_close.index)

display(df_ma200.iloc[201:205])

,MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,AFL,A,...,WMB,WTW,WDAY,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2025-02-20,122.483989,77.304112,110.688045,174.797849,335.607812,501.340149,145.49560,15.547573,100.052464,137.657148,...,47.199349,287.180105,244.634001,87.270659,60.303034,129.709251,132.516874,355.13015,108.188567,175.276146
2025-02-21,122.754793,77.226231,110.830991,175.015309,336.047054,501.230749,145.33565,15.510293,100.161624,137.657127,...,47.302930,287.550675,244.687901,87.265738,60.386451,129.685572,132.605334,355.17150,108.118697,175.233846
2025-02-24,123.003543,77.140589,110.982706,175.224409,336.367347,501.021449,145.13685,15.473852,100.264285,137.640477,...,47.400929,287.929774,244.690951,87.240239,60.470714,129.643969,132.689893,355.17880,108.038980,175.207432
2025-02-25,123.257505,77.052068,111.135838,175.448402,336.674355,500.775599,144.89850,15.437054,100.369140,137.617634,...,47.496528,288.314738,244.750451,87.207224,60.558582,129.596992,132.772837,355.14760,107.970424,175.210625


### Detecting Golden and Death Crosses in the last 14 days

In [148]:
def has_golden_cross(_ma50: np.ndarray, _ma200: np.ndarray) -> bool:
    # This is a boolean array with True where the 50-day moving average is greater than the 200-day moving average for a given day.
    ma50_gt_ma200 = _ma50 >= _ma200

    # We now check if there was a transition from False to True
    for i in range(1, len(ma50_gt_ma200)):
        prev = ma50_gt_ma200[i-1]
        current = ma50_gt_ma200[i]
        if not prev and current:
            return True

    return False

In [152]:
# Use the previously defined function to check for Golden Crosses in the last 14 days
stocks_with_golden_cross = []

for ticker in ticker_list:
    ma50 = df_ma50[ticker].values[-14:]
    ma200 = df_ma200[ticker].values[-14:]

    if has_golden_cross(ma50, ma200):
        stocks_with_golden_cross.append(ticker)

print(f'Found {len(stocks_with_golden_cross)} stocks with Golden Crosses in the last 14 days.')
print('First 10 stocks with Golden Crosses:', sorted(stocks_with_golden_cross)[:10])

Found 8 stocks with Golden Crosses in the last 14 days.
First 10 stocks with Golden Crosses: ['AMT', 'EW', 'KDP', 'MDLZ', 'MOH', 'NEM', 'SBAC', 'SJM']


In [147]:
# Create a function to check for Death Crosses, using the same approach as before

def has_death_cross(_ma50: np.ndarray, _ma200: np.ndarray) -> bool:
    # This is a boolean array with True where the 50-day moving average is greater than the 200-day moving average for a given day.
    ma50_gt_ma200 = _ma50 >= _ma200

    # We now check if there was a transition from True to False in the last `period` days.
    for i in range(1, len(ma50_gt_ma200)):
        prev = ma50_gt_ma200[i-1]
        current = ma50_gt_ma200[i]
        if prev and not current:
            return True

    return False

In [154]:
stocks_with_death_cross = []

for ticker in ticker_list:
    ma50 = df_ma50[ticker].values[-14:]
    ma200 = df_ma200[ticker].values[-14:]

    if has_death_cross(ma50, ma200):
        stocks_with_death_cross.append(ticker)

print(f'Found {len(stocks_with_death_cross)} stocks with Death Crosses in the last 14 days.')
print('First 10 stocks with Death Crosses:', sorted(stocks_with_death_cross)[:10])

Found 58 stocks with Death Crosses in the last 14 days.
First 10 stocks with Death Crosses: ['ACN', 'ADSK', 'AME', 'AMP', 'AMZN', 'APD', 'APO', 'AXP', 'BAC', 'BLK']


#### Golden crosses
List the first top companies in alphabetical order (by there symbol or ticker) that had a golden cross in the last 14 days:

- AMT
- EW
- KDP
- MDLZ
- MOH
- NEM
- SBAC
- SJM

#### Death crosses
List the first 10 companies in alphabetical order (by there symbol or ticker) that had a death cross in the last 14 days: 

- ACN
- ADSK
- AME
- AMP
- AMZN
- APD
- APO
- AXP
- BAC
- BLK

### Visualization of the results
(in alphabetical order)

#### Compute the volatility of every stock and print it in the title of each plot 

In [12]:
# CODE HERE
# Use as many coding cells as you need

# Compute the volatility of every stock in the S&P 500 


#### Plot top 10 stocks that had Golden Crosses in the last 14 days

- You should have 10 plots (use a for loop) for every stock in the top 10 (in alphabetical order)
- For each plot, put the volatility of the stock in the title of the plot

In [13]:
# CODE HERE
# Visualize the results here

### Plot top 10 stocks that had Death Crosses in the last 14 days

You should have 10 plots (use a for loop) for every stock in the top 10 (in alphabetical order)
For each plot, put the volatility of the stock in the title of the plot

In [14]:
# CODE HERE
# Visualize the results here

## Question section

### Understanding concepts

#### What is a Golden Cross and what does it typically signal to investors?

YOUR WRITTEN RESPONSE HERE



#### What is a Death Cross and how might market participants react to it?

YOUR WRITTEN RESPONSE HERE



#### Why might moving averages (MA50, MA200) be used as indicators in technical analysis?

YOUR WRITTEN RESPONSE HERE



#### Why are the last 14 days used to check for crosses? What are the implications of this choice?

YOUR WRITTEN RESPONSE HERE



#### How does volatility (e.g., measured using percentage change standard deviation) help contextualize the price movement around crosses?

YOUR WRITTEN RESPONSE HERE


### Backtesting and evaluation

#### How would you measure whether Golden Crosses actually lead to profitable trades?

YOUR WRITTEN RESPONSE HERE


#### What are the risks of using only technical indicators like moving averages without incorporating fundamentals?

YOUR WRITTEN RESPONSE HERE

#### How would you improve this strategy to reduce false signals (e.g., a Golden Cross that doesn’t lead to a price increase)?

YOUR WRITTEN RESPONSE HERE


### AI Integration


#### Could sentiment from news (future project part) help validate or invalidate these technical signals?

YOUR WRITTEN RESPONSE HERE


### Critical thinking

#### From a trading perspective, is this strategy actionable on its own?

YOUR WRITTEN RESPONSE HERE


#### Based on the volatility observed post-Golden Cross, do these crosses consistently predict upward movement?


YOUR WRITTEN RESPONSE HERE



#### If you had to present this analysis to a portfolio manager, what conclusions would you emphasize? What caveats would you include?

Conclusions in 2 bullet points:

YOUR WRITTEN RESPONSE HERE


Caveats in 2 or 3 bullet points:

YOUR WRITTEN REPSONSE HERE